## Homework

Implement a link prediction algorithm covered on the lecture (e.g., Common Neighbors, Jaccard's coefficient, Adamic/Adar). And try 2 more implemented in NetworkX (https://networkx.org/documentation/stable/reference/algorithms/link_prediction.html) on a network of your choice. Compare the results!

In [7]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

def get_missing_edges(G):
    existing_edges = set([frozenset(edge) for edge in G.edges()])
    possible_edges = set()
    for node1 in G.nodes():
        for node2 in G.nodes():
            if node1 != node2:
                possible_edges.add(frozenset((node1, node2)))
    return possible_edges - existing_edges

def common_neighbor_pred(G):
    scores = {}
    missing_edges = get_missing_edges(G)
    for edge in missing_edges:
        node1, node2 = tuple(edge)
        common_neighbors = len(list(nx.common_neighbors(G, node1, node2)))
        scores[edge] = common_neighbors
    return scores

def jaccard_coefficient_pred(G):
    scores = {}
    missing_edges = get_missing_edges(G)
    for edge in missing_edges:
        node1, node2 = tuple(edge)
        neighbors1 = set(G.neighbors(node1))
        neighbors2 = set(G.neighbors(node2))
        intersection = len(neighbors1 & neighbors2)
        union = len(neighbors1 | neighbors2)
        jaccard_score = intersection / union if union != 0 else 0
        scores[edge] = jaccard_score
    return scores

def adamic_adar_pred(G):
    scores = {}
    missing_edges = get_missing_edges(G)
    for edge in missing_edges:
        node1, node2 = tuple(edge)
        common_neighbors = list(nx.common_neighbors(G, node1, node2))
        adamic_adar_score = sum(1 / np.log(G.degree(neighbor)) for neighbor in common_neighbors if G.degree(neighbor) > 1)
        scores[edge] = adamic_adar_score
    return scores


In [8]:
import random

class KumpulaModel:
    def __init__(self, N, num_steps, pr, pt, pd, delta_w, w0=1.0):
        self.N = N
        self.num_steps = num_steps
        self.pr = pr
        self.pt = pt
        self.pd = pd
        self.delta_w = delta_w
        self.w0 = w0

        self.create_graph()
    
    def create_graph(self):
        self.G = nx.Graph()
        self.G.add_nodes_from(range(self.N))

        self.init_GA()

        for step in range(self.num_steps):
            self.LA_step()
            self.LD_step()
    
    def GA_step(self, i, j, p):
        if random.random() <= p:
            self.G.add_edge(i, j, weight=self.w0)

    def init_GA(self):
        for i in range(len(self.G.nodes)):
            for j in range(i+1, len(self.G.nodes)):
                self.GA_step(i, j, self.pr)
    
    def select_neighbor_weighted(self, node, neighbors):
        neighbor_weights = [self.G[node][nbr]['weight'] for nbr in neighbors]
        total_weight = sum(neighbor_weights)
        probabilities = [w / total_weight for w in neighbor_weights]
        return random.choices(neighbors, weights=probabilities)[0]

    def LA_step(self):
        i = random.choice(list(self.G.nodes()))
        i_neighbors = list(self.G.neighbors(i))

        if len(i_neighbors) == 0:
            j = random.choice(list(set(self.G.nodes) - {i}))
            self.GA_step(i, j, 1.0)
            return
        
        j = self.select_neighbor_weighted(i, i_neighbors)
        self.G[i][j]['weight'] += self.delta_w
        j_neighbors = list(self.G.neighbors(j))
        j_neighbors = [nbr for nbr in j_neighbors if nbr != i]

        if len(j_neighbors) == 0:
            k = random.choice(list(set(self.G.nodes) - {i, j}))
            self.GA_step(j, k, 1.0)
            return

        k = self.select_neighbor_weighted(j, j_neighbors)
        self.G[j][k]['weight'] += self.delta_w
        if not self.G.has_edge(i, k):
            self.GA_step(i, k, self.pt)
        else:
            self.G[i][k]['weight'] += self.delta_w
    
    def LD_step(self):
        if random.random() < self.pd:
            node = random.choice(list(self.G.nodes()))
            edges_to_remove = list(self.G.edges(node))
            self.G.remove_edges_from(edges_to_remove)



In [ ]:
N = 100
num_steps = 200*N
pr = 0.005
pt = 0.1
pd = 0.005
delta_w = 0.9

model = KumpulaModel(N, num_steps, pr, pt, pd, delta_w)
G = model.G

# nx functions
preferential_attachment_scores = nx.preferential_attachment(G)
common_neighbor_centrality_scores = nx.common_neighbor_centrality(G)

print("Top 5 Preferential Attachment Scores:")
for u, v, score in sorted(preferential_attachment_scores, key=lambda x: x[2], reverse=True)[:5]:
    print(f"Edge: ({u}, {v}), Score: {score}")

print("\nTop 5 Common Neighbor Centrality Scores:")
for u, v, score in sorted(common_neighbor_centrality_scores, key=lambda x: x[2], reverse=True)[:5]:
    print(f"Edge: ({u}, {v}), Score: {score}")

# own functions
common_neighbor_scores = common_neighbor_pred(G)
jaccard_scores = jaccard_coefficient_pred(G)
adamic_adar_scores = adamic_adar_pred(G)

cn_sorted = sorted(common_neighbor_scores.items(), key=lambda x: x[1], reverse=True)
jaccard_sorted = sorted(jaccard_scores.items(), key=lambda x: x[1], reverse=True)
aa_sorted = sorted(adamic_adar_scores.items(), key=lambda x: x[1], reverse=True)

print("\nTop 5 Common Neighbor Predictions:")
for edge, score in cn_sorted[:5]:
    print(f"Edge: {edge}, Score: {score}")

print("\nTop 5 Jaccard Coefficient Predictions:")
for edge, score in jaccard_sorted[:5]:
    print(f"Edge: {edge}, Score: {score}")

print("\nTop 5 Adamic-Adar Predictions:")
for edge, score in aa_sorted[:5]:
    print(f"Edge: {edge}, Score: {score}")

Top 5 Preferential Attachment Scores:
<generator object _apply_prediction.<locals>.<genexpr> at 0x0000021260897040>
Edge: (8, 60), Score: 210
Edge: (60, 75), Score: 210
Edge: (8, 69), Score: 196
Edge: (8, 75), Score: 196
Edge: (69, 75), Score: 196

Top 5 Common Neighbor Centrality Scores:
Edge: (8, 16), Score: 14.799999999999999
Edge: (19, 93), Score: 14.799999999999999
Edge: (37, 57), Score: 14.799999999999999
Edge: (48, 65), Score: 14.799999999999999
Edge: (51, 75), Score: 14.799999999999999
Top 5 Common Neighbor Predictions:
Edge: frozenset({19, 93}), Score: 6
Edge: frozenset({48, 65}), Score: 6
Edge: frozenset({72, 83}), Score: 6
Edge: frozenset({57, 37}), Score: 6
Edge: frozenset({75, 51}), Score: 6

Top 5 Jaccard Coefficient Predictions:
Edge: frozenset({31, 23}), Score: 1.0
Edge: frozenset({25, 14}), Score: 0.75
Edge: frozenset({73, 17}), Score: 0.75
Edge: frozenset({49, 6}), Score: 0.7142857142857143
Edge: frozenset({30, 63}), Score: 0.7142857142857143

Top 5 Adamic-Adar Predic